In [15]:
import cv2
import numpy as np

def process_street_image_optimized(image_path):
    
    image = cv2.imread(image_path)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive histogram equalization to improve contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized = clahe.apply(gray)

    # Apply a more aggressive Gaussian blur to smooth out noise
    blurred = cv2.GaussianBlur(equalized, (9, 9), 0)

    # Canny edge detection with adjusted thresholds to decrease noise
    edges = cv2.Canny(blurred, 50, 150)

    # Hough lines detection with parameters adjusted to filter out short lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=20)

    # Drawing the new lines on the original image
    line_image = np.copy(image)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Save the image with lines drawn on it
    result_path = image_path.replace('.jpg', '_optimized.jpg')
    cv2.imwrite(result_path, line_image)

    return result_path

street_image_path_3 = 'street3.jpg'


processed_optimized_path_3 = process_street_image_optimized(street_image_path_3)
processed_optimized_path_3

'street3_optimized.jpg'